In [1]:
# 필요한것 import 
import pandas as pd
import numpy as np

# 경사하강법(단층퍼셉트론)
from sklearn.linear_model import SGDClassifier

# 스케일러
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder

# 독립변수 탐색
import statsmodels.api as sm 

# 결측치 탐지
import missingno as msno

# tensorflow
from tensorflow import keras
# 그리스 아저씨
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split, cross_validate

# 시각화
import matplotlib.pyplot as plt

# 형태소 분석기 ----------------------------------------------
# 설정할 것 많음
# 1. 자바 일단 깔아서 program_files/JAVA 폴더 안에 넣기
# 2. 환경 변수 설정 JAVA_HOME :  폴더경로
# 3. pip install 2개... 
# 4. 형태소 객체실행해보기 ( 만약 안되면 주피터 다끄고 다시 실행 )
# from konlpy.tag import Okt
# from tensorflow.keras.preprocessing.text import Tokenizer

# 정규표현식
# import re 

# 데이터 불러오기
- 케라스 데이터
- csv, txt 데이터
- train, test 데이터

In [18]:
# 데이터 불러오기 (예시)
(x_train, y_train), (x_test, y_test) = keras.datasets.fashion_mnist.load_data()

# 전체 데이터 확인하기

- shape 찍어보기

# 종속변수 분포 확인하기
- 종속변수 비율, 컬럼수 등 확인
- 시각화

In [3]:
# np.unique(y_train, return_counts = True)

# **전처리**

In [ ]:
# 데이터 타입 확인

- 범주형, 연속형으로 변경

# **함수**

### 데이터 스케일링 함수

In [15]:
# 스케일링 하기
def img_scaled_data(data, w= None ,h = None, c = 1):
    # 근데 이것도... 3차원인지 4차원인지 구별해야함...ㅠㅠ
    data = data.reshape(-1, w, h, c) / 255
    return data

# 기본으로 stardscaler 쓸것임

# train 데이터만 fit 해줄것임
def scaled_data(data, train_data = False):
    ss = StandardScaler()
    if train_data:
        data = ss.fit_transform(data)
    else:
        data = ss.transform(data)

    return data

### 데이터 분할 함수

In [14]:
# 독립변수랑 종속변수 넣어줌됨
def split_data(x, y, test_size, random_state):
    # 데이터 나누기
    x_train, x_test, y_train, y_test = train_test_split(
        x,
        y, 
        test_size=test_size,
        stratify= y,
        random_state=random_state
    )
    print("나눈데이터 shape 확인")
    
    print(f"x train 데이터 {x_train.shape}")
    print(f"x test 데이터 {x_test.shape}")
    
    print(f"y train 데이터 {y_train.shape}")
    print(f"y test 데이터 {y_test.shape}")
    
    return x_train, x_test, y_train, y_test

### 성능 시각화 함수

In [13]:
def show_model(history):
    plt.figure()
    
    plt.plot(history.history["loss"], label = "train_loss")
    plt.plot(history.history["val_loss"], label = "val_loss")
    
    plt.plot(history.history["accuracy"], label = "train_accuracy")
    plt.plot(history.history["val_accuracy"], label = "val_accuracy")
    
    plt.legend()
    plt.xlabel("epoch")
    plt.show()

In [ ]:
# 모든 전처리가 끝나고 실행해야함
def my_model(x_train, x_test, y_train, y_test, img = False, binary = False):
    
    print("--------------- 데이터 확인---------------")
    print("독립 변수 확인")
    print("x_train.shape")
    print(x_train.shape)

    print("x_test.shape")
    print(x_test.shape)
    
    print("종속 변수 확인")
    print("y_train.shape")
    print(y_train.shape)
    
    print("y_test.shape")
    print(y_test.shape)


    print("종속변수 범주확인")
    print(np.unique(y_train, return_counts = True))

    category_length = len(np.unique(y_train, return_counts = True)[0])
    
    print("--------------- 데이터 확인 끝 ---------------")

    # 출력값 설정
    outputs = category_length
    
    # 스케일링
    # 이미지일경우
    if img :
        # 이미지의 너비 높이, 채널(색상수) 설정하기
        w = 28
        h = 28
        c = 1
        x_train = img_scaled_data(x_train, w = w , h = h, c = c)
        x_test = img_scaled_data(x_test, w = w , h = h, c = c)

    else: # 단위를 평준화 하기 위해 스케일링
        x_train = scaled_data(x_train, True)
        x_test = scaled_data(x_test)

    # 데이터 나누기
    x_sub, x_val, y_sub, y_val = split_data(x_train, y_train, 0.2, 34)
    
    print("--------------- 모델 만들기 ---------------")
    # 모델 만들기
    model = keras.Sequential()
    if img :
        model.add(keras.Input(shape = (w,h,c)))
    
    
        model.add(keras.layers.Conv2D(32, 3, activation="relu", padding= "same"))
        model.add(keras.layers.MaxPool2D(2))
    
        model.add(keras.layers.Conv2D(64, 3, activation="relu", padding= "same"))
        model.add(keras.layers.MaxPool2D(2))
    
    
        # 1차원으로 변경
        model.add(keras.layers.Flatten())
        
        model.add(keras.layers.Dense(100, activation = "relu"))
        model.add(keras.layers.Dropout(0.4))

    else :
        model.add(keras.Input(shape = (15,))) # 어떤 입력층이 들어갈지는 모르니께...
        
    # 출력층
    if binary: # 이진 분류일경우 
        model.add(keras.layers.Dense(1, activation = "sigmoid"))
    else: # 범주형 변수일경우
        model.add(keras.layers.Dense(outputs, activation = "softmax"))


    # 모델 요약보기(보이는지 확인해야함)
    model.summary()

    
    # 모델 설정
    if binary:
        loss = "binary_crossentropy"
    else:
        loss = "sparse_categorical_crossentropy"
        
    model.compile(loss = loss, optimizer = "adam", metrics = ["accuracy"])
    
    # 얼리스탑 콜백
    es_cb = keras.callbacks.EarlyStopping(patience=4, restore_best_weights= True)
    # 저장
    cp_cb = keras.callbacks.ModelCheckpoint("./model/이경민.keras", save_best_only=True)
    
    # 적용
    history = model.fit(
        x_sub, 
        y_sub, 
        epochs=100, 
        validation_data=(x_val, y_val), 
        callbacks=[es_cb], 
        batch_size= 32
    )


    print("====================시각화====================")
    show_model(history)
    
    return model


In [ ]:
# 테스트 해봐야함
model = my_model(x_train, x_test , y_train, y_test, img = True, binary=False )

In [12]:
# 일반화 성능 ---------
x_test = img_scaled_data(x_test, w = 28 , h = 28, c = 1)
model.evaluate(x_test, y_test)

313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9148 - loss: 0.2422   


[0.2422395944595337, 0.9147999882698059]

### 포켓몬 배틀 시작